In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
groqApiKey = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model = "gemma2-9b-it",
    groq_api_key = groqApiKey
)

model

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(
    content = "Hi, My name is Keshav and I am recent graduate from Boston University, specialising in AI"
)])

In [ ]:
## Hardcoding Texts

from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content = "Hi, My name is Keshav and I am recent graduate from Boston University, specialising in AI"),
    AIMessage(content = "Hi Keshav,\n\nThat's great to hear! Congratulations on graduating from Boston University with a specialization in AI.  \n\nWhat are your plans now that you've finished your studies? Are you looking for a job in the field, or perhaps pursuing further research?\n\nI'm always interested to learn about the work that people are doing in AI.  \n\n"),
    HumanMessage(content = "Hey, what is my name and where did I recently graduate from?")
])


## Message History

In [ ]:
from langchain_community.chat_message_histories         import ChatMessageHistory
from langchain_core.chat_history                        import BaseChatMessageHistory
from langchain_core.runnables.history                   import RunnableWithMessageHistory


## Get Session History
##    @input    - sessionId:str         -> unique session id
##    @output   - ChatMessageHistory    -> BaseChatHistory

ids = {}

def getSessionHistory(sessionId: str): 
    if sessionId not in ids:
        ids[sessionId] = ChatMessageHistory()
    return ids[sessionId]

withMessageHistory = RunnableWithMessageHistory(model, getSessionHistory)

In [ ]:
## Hardcoding Values

config = {"configurable": {"session_id": "chat1"}}

In [ ]:
response = withMessageHistory.invoke([
    HumanMessage(content = "Hi, My name is Keshav and I am recent graduate from Boston University, specialising in AI.")],
    config = config
)

In [ ]:
response.content

In [ ]:
# change in config with sessionId to test the response

configOne = {"configurable": {"session_id": "chat1"}}
configTwo = {"configurable": {"session_id": "chat2"}}

response = withMessageHistory.invoke([
    HumanMessage(content = "What is my name?")],
    config = configTwo
)

In [ ]:
response.content

## Prompt Template

In [ ]:
from langchain_core.prompts         import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability."),
    MessagesPlaceholder(variable_name = "messages")
])

In [ ]:
chain = prompt|model

In [ ]:
chain.invoke({
    "messages": [HumanMessage(content = "Hi, my name is Keshav Saraogi.")]
})

In [ ]:
withMessageHistory = RunnableWithMessageHistory(chain, getSessionHistory)

In [ ]:
config = {"configurable": {"session_id": "chatThree"}}
response = withMessageHistory.invoke(
    [HumanMessage(content = "Hi, My name is Keshav")],
    config = config
)

response.content

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."), 
    MessagesPlaceholder(variable_name = "messages")
])

chain = prompt|model

In [ ]:
response = chain.invoke(
    {
    "messages": [HumanMessage(content = "Hi, My name is Keshav")],
    "language": "Hindi"
    }
)

response.content

## Message History

In [ ]:
withMessageHistory = RunnableWithMessageHistory(
    chain, 
    getSessionHistory,
    input_messages_key = "messages"
)

In [ ]:
config = {"configurable": {"session_id": "chatFour"}}

response = withMessageHistory.invoke({
    'messages': [HumanMessage(content = "Hi, I am Keshav Saraogi")],
    "language": "Hindi" 
    },
    config = config
)

In [ ]:
response.content

In [ ]:
response = withMessageHistory.invoke({
    'messages': [HumanMessage(content = "What is my name?")],
    "language": "Hindi" 
    },
    config = config
)

response.content

## Conversation History